In [2]:
%matplotlib qt

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pickle


In [3]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [11]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder

In [6]:
football = pd.read_csv('data/england-transformed.csv')
football = football[football.season >= 2007]
football.head()

,season,week_day,year,month,day,home,visitor,home_current_points,visitor_current_points,home_last_table_position,visitor_last_table_position,home_penultimate_table_position,visitor_penultimate_table_position,result
11202,2007,1,2007,8,11,2,24,0,3,11,3,16,3,3
11203,2007,1,2007,8,11,7,30,0,3,7,13,8,7,3
11204,2007,1,2007,8,11,16,36,1,1,21,9,21,17,1
11205,2007,1,2007,8,11,17,50,3,0,6,18,11,10,2
11206,2007,1,2007,8,11,28,5,0,3,12,10,14,6,3


In [7]:
X = football.iloc[:, :-1].to_numpy()
y = football.iloc[:, -1].to_numpy()

In [21]:
one_X = OneHotEncoder(drop='first').fit_transform(X).toarray()
one_X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [24]:
pca = PCA(n_components=2).fit_transform(one_X)


TypeError: loop of ufunc does not support argument 0 of type NoneType which has no callable sqrt method

In [13]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
pca = PCA(n_components=3).fit_transform(X)

In [10]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(pca[:, 0], pca[:, 1], pca[:, 2], c = y)
ax.set_ylabel("Y−axis")
ax.set_xlabel("X−axis")
ax.set_zlabel("Z−axis")
plt.show()

In [17]:
tcsv = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tcsv.split(one_X):
    #print(f'Train idx: {train_index}, Test idx: {test_index}')
    X_train, X_test = one_X[train_index], one_X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = GaussianNB().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, labels=[1,2,3]))

              precision    recall  f1-score   support

           1       0.25      0.41      0.31       114
           2       0.55      0.28      0.38       215
           3       0.31      0.38      0.34       120

    accuracy                           0.34       449
   macro avg       0.37      0.36      0.34       449
weighted avg       0.41      0.34      0.35       449

              precision    recall  f1-score   support

           1       0.32      0.82      0.46       130
           2       0.74      0.08      0.14       213
           3       0.28      0.25      0.26       106

    accuracy                           0.33       449
   macro avg       0.45      0.38      0.29       449
weighted avg       0.51      0.33      0.26       449

              precision    recall  f1-score   support

           1       0.26      0.78      0.39       116
           2       0.52      0.08      0.14       206
           3       0.31      0.16      0.21       127

    accuracy        

In [19]:
tcsv = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tcsv.split(one_X):
    #print(f'Train idx: {train_index}, Test idx: {test_index}')
    X_train, X_test = one_X[train_index], one_X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = KNeighborsClassifier(n_neighbors=13).fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, labels=[1,2,3]))

              precision    recall  f1-score   support

           1       0.26      0.05      0.09       114
           2       0.50      0.73      0.59       215
           3       0.37      0.36      0.37       120

    accuracy                           0.46       449
   macro avg       0.38      0.38      0.35       449
weighted avg       0.41      0.46      0.40       449

              precision    recall  f1-score   support

           1       0.28      0.08      0.13       130
           2       0.49      0.74      0.59       213
           3       0.29      0.24      0.26       106

    accuracy                           0.43       449
   macro avg       0.35      0.35      0.33       449
weighted avg       0.38      0.43      0.38       449

              precision    recall  f1-score   support

           1       0.22      0.18      0.20       116
           2       0.48      0.72      0.58       206
           3       0.33      0.13      0.18       127

    accuracy        

In [18]:
tcsv = TimeSeriesSplit(n_splits=10)
for train_index, test_index in tcsv.split(one_X):
    #print(f'Train idx: {train_index}, Test idx: {test_index}')
    X_train, X_test = one_X[train_index], one_X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = LogisticRegression(multi_class='multinomial').fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred, labels=[1,2,3]))

              precision    recall  f1-score   support

           1       0.25      0.17      0.20       114
           2       0.61      0.48      0.54       215
           3       0.37      0.62      0.46       120

    accuracy                           0.44       449
   macro avg       0.41      0.42      0.40       449
weighted avg       0.45      0.44      0.43       449

              precision    recall  f1-score   support

           1       0.32      0.28      0.30       130
           2       0.55      0.62      0.58       213
           3       0.29      0.26      0.27       106

    accuracy                           0.43       449
   macro avg       0.39      0.39      0.38       449
weighted avg       0.42      0.43      0.43       449

              precision    recall  f1-score   support

           1       0.27      0.26      0.26       116
           2       0.52      0.57      0.54       206
           3       0.42      0.36      0.39       127

    accuracy        

In [204]:
model = KNeighborsClassifier(n_neighbors=15).fit(X, y)
pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)